In [9]:
import numpy as np

parser_version = '1.6.7'
!java -cp "document-parser-$parser_version/classes;document-parser-$parser_version/lib/*" com.nemo.document.parser.App -i "Документы\ДД по практикам\Практика правового сопровождения закупок МТР и услуг общего профиля\ДД ООО И.doc"

04:05:39.615 [main] ERROR com.nemo.document.parser.App - Error: 
java.io.FileNotFoundException: Документы\ДД по практикам\Практика правового сопровождения закупок МТР и услуг общего профиля\ДД ООО И.doc (Системе не удается найти указанный путь)
	at java.base/java.io.FileInputStream.open0(Native Method)
	at java.base/java.io.FileInputStream.open(FileInputStream.java:216)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:157)
	at com.nemo.document.parser.DocumentParser.parse(DocumentParser.java:95)
	at com.nemo.document.parser.App.main(App.java:29)


# Start

## Import

In [3]:
import platform
import sys
import os
import json
import subprocess
import pandas as pd
import fnmatch
import base64
import requests
import time
import importlib
import logging
import search_text
import search_text_v2
import signal
import numpy as np
from pathlib import Path

## Create list of all docs path

In [11]:
arrOfAllDocs = []

for root, dir_names, filenames in os.walk('Документы'):
    if len(root.split('\\')) == 2: continue
    flag = False
    for i in root.split('\\'):
        if str(i).startswith('Исключена'):
            flag = True
            break
    if flag: continue
    for filename in fnmatch.filter(filenames, '*.docx'):
        arrOfAllDocs.append(os.path.join(root, filename))
    for filename in fnmatch.filter(filenames, '*.doc'):
        arrOfAllDocs.append(os.path.join(root, filename))


In [12]:
arrOfAllDocs[:]

['Документы\\Входящие\\Практика коммерческой логистики\\01-Шаблон Исковое заявление Типовой договор поставки ГПН (штраф за простой вагонов).docx',
 'Документы\\Входящие\\Практика коммерческой логистики\\Акт общей формы_ШАБЛОН_ЛЕД_дополнен.docx',
 'Документы\\Входящие\\Практика коммерческой логистики\\АКТ ОТБОРА ПРОБ_ШАБЛОН.docx',
 'Документы\\Входящие\\Практика коммерческой логистики\\АКТ ПРИЕМКИ по КАЧЕСТВУ_ШАБЛОН.docx',
 'Документы\\Входящие\\Практика коммерческой логистики\\ПРЕТЕНЗИЯ_излишки ГПН - ГПН-РП_ШАБЛОН для Регламента.docx',
 'Документы\\Входящие\\Практика коммерческой логистики\\ПРЕТЕНЗИЯ_недостача ГПН - ГПН-РП_ШАБЛОН для Регламента.docx',
 'Документы\\Входящие\\Практика коммерческой логистики\\расчет претензии ГПН-Л к РЖД_ШАБЛОН.docx',
 'Документы\\Входящие\\Практика коммерческой логистики\\Соглашение о зачете встречных однородных_ШАБЛОН.docx',
 'Документы\\Входящие\\Практика коммерческой логистики\\ШАБЛОН письма контрагенту о перерасчете требований по претензии.docx',
 'Д

In [13]:
import multiprocessing


def driver_func(documents, path):
    processes = 4
    with multiprocessing.Pool(processes) as pool:
        params = zip(documents, path)
        print(params)
        results = [pool.apply_async(search_text_v2.get_text, param) for param in params]

        for r in results:
            print('\t', r.get())

In [14]:
importlib.reload(search_text)
importlib.reload(search_text_v2)

index = 1
number_of_docs = 0
parser_version = '1.6.7'

labels = ['Практика коммерческой логистики',
          'Практика недропользования и экологии',
          'Практика поддержки региональных, розничных продаж и клиентского сервиса',
          'Практика правового сопровождения закупок МТР и услуг общего профиля',
          'Практика правового сопровождения земельных отношений и сделок с недвижимым имуществом',
          'Практика правового сопровождения операционной деятельности БРД',
          'Практика правового сопровождения переработки и инфраструктуры',
          'Практика правовой поддержки брендов',
          'Практика правовой поддержки использования и коммерциализации ИС',
          'Практика правовой поддержки создания и приобретения ИС',
          'Практика промышленной безопасности и охраны труда',
          'Практика финансового и конкурентного права',
          'Практика экспорта, оптовых продаж и сбыта бизнес-единиц (БЕ)']

result = []
result_of_fail = []
result_of_possible = []
result_of_possible2 = []

s = [
    "java",
    "-jar",
    f"document-parser-{parser_version}.jar",
    "--server.port=8083"
]
headers = {
    'Content-type': 'application/json',
    'Accept': 'application/json; text/plain'
}

parser_url = "http://localhost:8083/"
java_subprocess = None
try:
    i = 1
    try:
        print(f"Проверка, если парсер заупущен на {parser_url}")
        response = requests.post(
            f"{parser_url}status",
            headers=headers
        )
    except Exception as e:
        print(
            "Запуск document-parser на 8083 порту, если что-то пойдет не так, то руками УБЕЙТЕ java процесс"
        )
        java_subprocess = subprocess.Popen(s, creationflags=subprocess.CREATE_NEW_PROCESS_GROUP,
                                           stdout=subprocess.PIPE, encoding="utf-8")
        time.sleep(2)
        while True:
            # time.sleep(0.1)
            output_log_spring = java_subprocess.stdout.readline()
            sys.stdout.write("\rПроверка соединения")
            sys.stdout.flush()
            i += 1
            if output_log_spring.find("Started DocumentParserService") != -1:
                print("\nГотово")
                java_subprocess.stdout.close()
                break
            if i > 31:
                raise Exception("Не удалось получить доступ к ранее запущенному парсеру")
    print("Запустился успешно")
    print("Общее количество документов =", len(arrOfAllDocs))

    for docs in arrOfAllDocs:
        try:
            file = open(docs, 'rb')
            encoded_string = base64.b64encode(file.read())
            encoded_string = str(encoded_string)[2:-1]
        except Exception as e:
            # print(f"\nОшибка в файле {docs}")
            # print(f"при конвертации в base64, исключение = {e.msg}")
            # print("=" * 200)
            continue
        is_doc = True
        is_docx = True
        is_not_bad_doc = True
        doc_type = docs.split(".")[-1].upper()
        resArr = []
        document = []
        while is_doc or is_docx or is_not_bad_doc:
            response = requests.post(
                f"{parser_url}document-parser",
                data=json.dumps({
                    "base64Content": encoded_string,
                    "documentFileType": doc_type
                }),
                headers=headers
            )
            if 'message' in response.json() and not (is_doc == False and is_docx == False):
                if doc_type == 'DOC':
                    is_doc = False
                    doc_type = 'DOCX'
                    continue
                if doc_type == 'DOCX':
                    is_docx = False
                    doc_type = 'DOC'
                    continue

            try:
                resArr = response.json()['documents']
            except Exception as e:
                # print(f"\nОшибка в файле {docs}")
                # print(f"Ответ от парсера {response.json()}")
                # print("=" * 200)
                continue
            finally:
                is_doc = False
                is_docx = False
                is_not_bad_doc = False

        sys.stdout.write(
            f"\rNow {index} from {len(arrOfAllDocs)} и Good = {len(result)}, Bad = {len(result_of_fail)}, TEST = {len(result_of_possible)}, TEST2 = {len(result_of_possible2)}")
        sys.stdout.flush()
        index += 1

        if not resArr:
            continue
        # driver_func(resArr, docs)
        for document in resArr:
            # result_of_single_doc, enum = search_text.find_text(document, path=docs)
            result_of_single_doc, enum = search_text_v2.get_text(document, path=docs)

            if enum == search_text_v2.list_of_sheets.GOOD:
                result.append(result_of_single_doc)
            if enum == search_text_v2.list_of_sheets.BAD:
                result_of_fail.append(result_of_single_doc)
            if enum == search_text_v2.list_of_sheets.BAD2:
                result_of_possible.append(result_of_single_doc)
            if enum == search_text_v2.list_of_sheets.GOOD2:
                result.append(result_of_single_doc)
                result_of_possible2.append(result_of_single_doc)

    with pd.ExcelWriter("classifier.xlsx", engine="xlsxwriter",
                        engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
        # writer = pd.ExcelWriter("classifier.xlsx", engine='xlsxwriter')

        df = pd.DataFrame(result)
        df = df.drop_duplicates(subset=['text'])
        df.to_excel(writer, 'good', engine='xlsxwriter')
        sheets_good = writer.sheets['good']
        sheets_good.autofilter(0, 0, df.shape[0], df.shape[1])

        df = pd.DataFrame(result_of_fail)
        df.to_excel(writer, 'bad', engine='xlsxwriter')
        sheets_bad = writer.sheets['bad']
        sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])
        logging.info("\nФайл создан")
except Exception as e:
    logging.error(e)
finally:
    #Смерть java процессу!
    if java_subprocess is not None:
        if platform.system() == 'Windows':
            subprocess.run("TASKKILL /F /PID {pid} /T".format(pid=java_subprocess.pid))
        elif platform.system() == 'Linux':
            os.kill(java_subprocess.pid, signal.SIGTERM)
        else:
            print('Не известная платформа, убейте в ручную процесс java')
# Now 2277 from 2277 и Good = 2970, Bad = 52, TEST = 846, TEST2 = 2053

Проверка, если парсер заупущен на http://localhost:8083/
Запуск document-parser на 8083 порту, если что-то пойдет не так, то руками УБЕЙТЕ java процесс
Проверка соединения
Готово
Запустился успешно
Общее количество документов = 2040
Now 2040 from 2040 и Good = 3402, Bad = 80, TEST = 0, TEST2 = 0

In [54]:
df = pd.read_excel(open('Фразы для Классификатора.xlsx', 'rb'), sheet_name='Лист1', skiprows=1)

In [55]:
df = df.drop(df.columns[[3, 4]], axis=1)
df = df.rename(columns={df.columns[0]: "path", df.columns[1]: "name", df.columns[2]: "text"})
# df = df.index.names = ['path']
df.head(10)

,path,name,text
0,Практика коммерческой логистики,NaN,NaN
1,Договорные документы,NaN,NaN
2,1,20210517_Договор_-ГПНА_Перевозка ЧАО,2.1.\tВ соответствии с условиями настоящего До...
3,2,Договор автоперевоз 2021 с учетом замечаний,"1.1.\tПеревозчик в порядке и на условиях, уста..."
4,3,Прил.3 Сентябрь_4000_20210831,"1.\tЗаказчик передает, а Перевозчик принимает ..."
5,4,Приложение 3а Акт приема по количеству.,"Настоящий акт составлен в том, что при приеме ..."
6,5,Проект ДОГОВОР НА ПЕРЕВОЗКУ,"1.1.\tПеревозчик в порядке и на условиях, уста..."
7,6,ТИПОВОЙ Договор перевалки нефтепродуктов 1,1.1.\tВ соответствии с настоящим Договором Исп...
8,Входящие документы,NaN,NaN
9,1,01-Шаблон Исковое заявление Типовой договор по...,ИСКОВОЕ ЗАЯВЛЕНИЕ\nо взыскании штрафа по догов...


In [56]:
path = ''
for index, row in df.iterrows():
    if len(df) != index + 1 and not pd.isnull(row.str.isdigit()['path']) and not pd.isnull(
            df.iloc[index + 1].str.isdigit()['path']):
        path = "Фразы/" + df.iloc[index + 1]['path'] + '/' + row['path'] + '/'

    if pd.isnull(row.str.isdigit()['path']) and path:
        if len(df) != index + 1 and not pd.isnull(
                df.iloc[index + 1].str.isdigit()['path']):
            path = 'Фразы/' + df.iloc[index + 1]['path'] + '/' + Path(path).parts[2] + '/'

        df.at[index, 'path'] = path + str(row['name'])
df = df.dropna()
df = df.reset_index(drop=True)

In [57]:
unique_categories = df['path'].transform(lambda x: Path(x).parts[2]).unique()
unique_categories

array(['Практика коммерческой логистики',
       'Практика недропользования и экологии',
       'Практика поддержки региональных, розничных продаж и клиентского сервиса',
       'Практика правового сопровождения закупок МТР и услуг общего профиля',
       'Практика правового сопровождения земельных отношений и сделок с недвижимым имуществом',
       'Практика правового сопровождения операционной деятельности БРД',
       'Практика правового сопровождения переработки и инфраструктуры',
       'Практика правовой поддержки брендов',
       'Практика правовой поддержки использования и коммерциализации ИС',
       'Практика правовой поддержки создания и приобретения ИС',
       'Практика промышленной безопасности и охраны труда',
       'Практика финансового и конкурентного права',
       'Практика экспорта, оптовых продаж и сбыта бизнес-единиц'],
      dtype=object)

In [58]:
df_classifier = pd.read_excel(open('classifier.xlsx', 'rb'), sheet_name='good', index_col=0)
df_classifier_bad = pd.read_excel(open('classifier.xlsx', 'rb'), sheet_name='bad', index_col=0)

In [59]:
df_classifier['path'].transform(lambda x: Path(x).parent.parts[2]).unique()

array(['Практика коммерческой логистики',
       'Практика недропользования и экологии',
       'Практика правового сопровождения закупок МТР и услуг общего профиля',
       'Практика правового сопровождения земельных отношений и сделок с недвижимым имуществом',
       'Практика правового сопровождения операционной деятельности БРД',
       'Практика правовой поддержки создания и приобретения ИС',
       'Практика прововой поддержки брендов',
       'Практика промышленной безопасности и охраны труда',
       'Практика судедбной защиты',
       'Практика финансового и конкурентного права',
       'Практика экспорта, оптовых продаж и сбыта бизнес-едениц',
       'Практика поддержки региональных, розничных продаж и клиентского сервиса',
       'Практика правового сопровождения переработки и инфраструктуры',
       'Практика правовой поддержки брендов',
       'Практика правовой поддержки использования и коммерциализации ИС',
       'Практика судебной защиты',
       'Практика экспорта, оп

In [60]:
unique_categories_form_main_file = df_classifier['path'].transform(lambda x: Path(x).parent.parts[2]).unique()

[category for category in unique_categories if category in unique_categories_form_main_file]

['Практика коммерческой логистики',
 'Практика недропользования и экологии',
 'Практика поддержки региональных, розничных продаж и клиентского сервиса',
 'Практика правового сопровождения закупок МТР и услуг общего профиля',
 'Практика правового сопровождения земельных отношений и сделок с недвижимым имуществом',
 'Практика правового сопровождения операционной деятельности БРД',
 'Практика правового сопровождения переработки и инфраструктуры',
 'Практика правовой поддержки брендов',
 'Практика правовой поддержки использования и коммерциализации ИС',
 'Практика правовой поддержки создания и приобретения ИС',
 'Практика промышленной безопасности и охраны труда',
 'Практика финансового и конкурентного права',
 'Практика экспорта, оптовых продаж и сбыта бизнес-единиц']

In [61]:
concat = pd.concat([df_classifier, df[['path', 'text', 'name']]])
concat = concat.reset_index(drop=True)
concat.tail(100)

,path,documentType,name,text,length,characterLength,wordsLength
3811,Фразы/Входящие документы/Практика финансового ...,NaN,[Untitled],о признании недействующими индивидуальных тари...,NaN,NaN,NaN
3812,Фразы/Входящие документы/Практика финансового ...,NaN,123,в котором с учетом уточнения просит признать и...,NaN,NaN,NaN
3813,Фразы/Входящие документы/Практика финансового ...,NaN,001064,ИЗВЕЩЕНИЕ «Об установлении тарифов на транспор...,NaN,NaN,NaN
3814,Фразы/Входящие документы/Практика финансового ...,NaN,001726,заявление об установлении тарифов на услуги по...,NaN,NaN,NaN
3815,Фразы/Входящие документы/Практика финансового ...,NaN,001738,13.04.2021 ООО «Газпромнефть-Энергосервис» под...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
3906,Фразы/Распорядительные документы/Практика эксп...,NaN,Приказ Об отражении недостачи от 30.12.2021 г.,ПРИКАЗ Об отражении в учёте результатов приёмк...,NaN,NaN,NaN
3907,Фразы/Распорядительные документы/Практика эксп...,NaN,Приказ,"ПРИКАЗ О введении в действие ""Регламента взаим...",NaN,NaN,NaN
3908,"Фразы/Доверенности/Практика экспорта, оптовых ...",NaN,Приказ_Утв прогр компенсац 2022 год,Об утверждении Положения и основных требований...,NaN,NaN,NaN
3909,"Фразы/Доверенности/Практика экспорта, оптовых ...",NaN,начупр,связанные с исполнением заключенных Обществом ...,NaN,NaN,NaN


In [62]:
df_classifier[df_classifier['text'].apply(lambda x: not isinstance(x, str))]

,path,documentType,name,text,length,characterLength,wordsLength


In [63]:
with pd.ExcelWriter("classifier.xlsx", engine="xlsxwriter",
                    engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
    # writer = pd.ExcelWriter("classifier.xlsx", engine='xlsxwriter')

    concat.to_excel(writer, 'good', engine='xlsxwriter')
    sheets_good = writer.sheets['good']
    sheets_good.autofilter(0, 0, df.shape[0], df.shape[1])

    df_classifier_bad.to_excel(writer, 'bad', engine='xlsxwriter')
    sheets_bad = writer.sheets['bad']
    sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])
    print("\nФайл создан")


Файл создан
